In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 设置GPU环境
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# RNN模型定义
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # 初始化隐藏状态
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # 前向传播RNN
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]  # 获取最后一个时间步的输出

        # 前向传播全连接层
        out = self.fc(out)
        return out

# 参数设置
sequence_length = 28  # MNIST图像的行数
input_size = 28  # MNIST图像的列数
hidden_size = 128  # 隐藏层的大小
num_layers = 2  # RNN的层数
num_classes = 10  # MNIST总共有10个类别（0到9）

# 数据加载和预处理
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# 创建RNN模型，并将其移至GPU
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 1
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        images = images.squeeze(1).to(device)  # 去掉通道维度，并移至GPU
        labels = labels.to(device)

        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(trainloader)}], Loss: {loss.item():.4f}')

# 测试模型
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        images = images.squeeze(1).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Epoch [1/1], Step [100/938], Loss: 1.0332
Epoch [1/1], Step [200/938], Loss: 0.7066
Epoch [1/1], Step [300/938], Loss: 0.6790
Epoch [1/1], Step [400/938], Loss: 0.5706
Epoch [1/1], Step [500/938], Loss: 0.4196
Epoch [1/1], Step [600/938], Loss: 0.5854
Epoch [1/1], Step [700/938], Loss: 0.3667
Epoch [1/1], Step [800/938], Loss: 0.5391
Epoch [1/1], Step [900/938], Loss: 0.4249
Test Accuracy of the model on the 10000 test images: 91.52 %
